In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool


In [124]:
class CFG:
    SEED = 42

In [125]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

In [126]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class','father','mother','gender'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id','father','mother','gender'])
        return df_x

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)
test_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


In [127]:
train_y

0      B
1      C
2      B
3      A
4      C
      ..
257    B
258    C
259    A
260    A
261    B
Name: class, Length: 262, dtype: object

In [128]:
numerical_feats = train_x.dtypes[train_x.dtypes != "object"].index.tolist()
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train_x.dtypes[train_x.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  1
Number of Categorical features:  15


In [129]:
numerical_feats

['trait']

In [130]:
categorical_feats

['SNP_01',
 'SNP_02',
 'SNP_03',
 'SNP_04',
 'SNP_05',
 'SNP_06',
 'SNP_07',
 'SNP_08',
 'SNP_09',
 'SNP_10',
 'SNP_11',
 'SNP_12',
 'SNP_13',
 'SNP_14',
 'SNP_15']

In [131]:
encoder = OrdinalEncoder(categorical_feats)
train_x[categorical_feats] = encoder.fit_transform(train_x[categorical_feats])

test_x[categorical_feats] = encoder.transform(test_x[categorical_feats])

In [132]:
scaler = StandardScaler()
train_x[numerical_feats] = scaler.fit_transform(train_x[numerical_feats])
test_x[numerical_feats] = scaler.transform(test_x[numerical_feats])


In [133]:
train_y = encoder.fit_transform(train_y)
train_y

,class
0,1
1,2
2,1
3,3
4,2
...,...
257,1
258,2
259,3
260,3


In [134]:
test_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,-1.672454,2,2,1,1,2,2,2,2,2,2,1,2,2,3,3
1,0.597924,1,1,3,3,3,1,1,3,1,1,1,1,1,1,1
2,0.597924,1,1,1,2,1,2,1,3,1,2,2,2,2,1,2
3,0.597924,1,1,2,2,3,1,1,3,1,1,2,2,3,1,1
4,-1.672454,3,2,1,3,2,3,2,3,3,2,3,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0.597924,2,2,3,2,1,2,1,1,1,1,3,1,1,1,3
171,0.597924,1,3,1,2,1,2,1,3,1,2,2,1,3,1,3
172,0.597924,1,3,1,2,1,2,1,3,1,1,1,1,3,1,2
173,0.597924,2,2,2,1,3,3,1,2,1,1,1,1,1,1,1


In [135]:
X = train_x
y = train_y
X_test = test_x

In [118]:
model_cat = CatBoostClassifier()
model_cat.fit(train_x, train_y, early_stopping_rounds=100, verbose=100)

Learning rate set to 0.073902
0:	learn: 1.0294361	total: 1.05ms	remaining: 1.05s
100:	learn: 0.1150238	total: 95.6ms	remaining: 851ms
200:	learn: 0.0557576	total: 158ms	remaining: 630ms
300:	learn: 0.0348391	total: 256ms	remaining: 594ms
400:	learn: 0.0247550	total: 322ms	remaining: 481ms
500:	learn: 0.0189531	total: 387ms	remaining: 386ms
600:	learn: 0.0152756	total: 486ms	remaining: 323ms
700:	learn: 0.0127049	total: 549ms	remaining: 234ms
800:	learn: 0.0108346	total: 610ms	remaining: 152ms
900:	learn: 0.0094084	total: 688ms	remaining: 75.6ms
999:	learn: 0.0083562	total: 770ms	remaining: 0us


In [139]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train_x, train_y):
  folds.append((train_idx,valid_idx))


random.seed(42)
cat_models={}

cat_features =[0,1,2,5,6,7,8,15,18]

for fold in range(5):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['class'],axis=1).iloc[train_idx].values 
  X_valid = train.drop(['class'],axis=1).iloc[valid_idx].values
  y_train = train['class'][train_idx].values
  y_valid = train['class'][valid_idx].values

  cat = CatBoostClassifier()
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

====================================1============================================


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=9]="C A": Cannot convert 'b'C A'' to float

In [112]:
preds = model_cat.predict(test_x)
print('Done.')

print(preds)


Done.
[[3]
 [1]
 [2]
 [1]
 [3]
 [1]
 [2]
 [1]
 [3]
 [3]
 [2]
 [1]
 [2]
 [3]
 [1]
 [1]
 [3]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [3]
 [3]
 [1]
 [2]
 [3]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [3]
 [1]
 [3]
 [1]
 [1]
 [1]
 [2]
 [3]
 [1]
 [2]
 [3]
 [1]
 [2]
 [2]
 [2]
 [3]
 [2]
 [3]
 [3]
 [1]
 [1]
 [1]
 [3]
 [3]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [3]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [1]
 [3]
 [2]
 [3]
 [1]
 [1]
 [2]
 [3]
 [3]
 [2]
 [1]
 [3]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [3]
 [3]
 [2]
 [1]
 [2]
 [3]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [2]
 [3]
 [3]
 [1]
 [3]
 [3]
 [3]
 [2]
 [1]
 [1]
 [1]
 [3]
 [1]
 [2]
 [3]
 [3]
 [1]
 [3]
 [1]
 [1]
 [3]
 [3]
 [1]
 [2]
 [3]
 [1]
 [2]
 [2]
 [1]
 [3]
 [3]
 [2]
 [1]
 [1]
 [3]
 [1]
 [2]
 [2]
 [1]
 [1]]


In [116]:
test_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,-1.672454,2,2,1,1,2,2,2,2,2,2,1,2,2,3,3
1,0.597924,1,1,3,3,3,1,1,3,1,1,1,1,1,1,1
2,0.597924,1,1,1,2,1,2,1,3,1,2,2,2,2,1,2
3,0.597924,1,1,2,2,3,1,1,3,1,1,2,2,3,1,1
4,-1.672454,3,2,1,3,2,3,2,3,3,2,3,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0.597924,2,2,3,2,1,2,1,1,1,1,3,1,1,1,3
171,0.597924,1,3,1,2,1,2,1,3,1,2,2,1,3,1,3
172,0.597924,1,3,1,2,1,2,1,3,1,1,1,1,3,1,2
173,0.597924,2,2,2,1,3,3,1,2,1,1,1,1,1,1,1


In [106]:
clf = RandomForestClassifier(random_state=CFG.SEED)
clf.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

In [107]:
preds = clf.predict(test_x)
print('Done.')

print(preds)


Done.
[3 1 2 1 3 1 2 1 3 3 2 1 1 3 1 1 3 1 1 2 1 1 1 3 1 2 1 3 3 1 3 3 1 2 3 1 2
 1 1 2 3 1 2 1 1 1 1 2 1 2 3 1 3 1 1 1 2 3 1 2 3 1 2 2 2 3 2 3 3 1 1 1 3 3
 2 1 2 1 1 1 2 1 3 1 1 1 1 1 2 3 1 1 2 1 1 2 3 1 3 2 3 1 1 2 3 3 2 1 3 1 2
 1 1 1 1 3 3 2 1 1 3 1 1 2 2 1 2 1 3 1 3 3 1 1 1 2 3 3 1 3 3 3 2 1 1 1 3 1
 2 3 3 1 3 1 1 3 3 1 2 3 1 2 2 1 3 3 2 1 1 3 1 2 2 1 1]


KeyError: 'class'

In [117]:
submit = pd.read_csv('./sample_submission.csv')

submit['class'] = encoder.inverse_transform(preds)

submit.to_csv('./answer/submit.csv', index=False)

KeyError: 'class'